In [13]:
import argparse
import os
from importlib import import_module
import numpy as np
import random
import pandas as pd
import torch
from torch.utils.data import DataLoader

from dataset import TestDataset, MaskBaseDataset

In [14]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [15]:
seed_everything(525)

In [16]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Models

In [17]:
def load_model(model_path, model_name, num_classes, device):
    model_cls = getattr(import_module("model"), model_name)
    model = model_cls(
        num_classes=num_classes
    )

    model_path = os.path.join(model_path)
    model.load_state_dict(torch.load(model_path, map_location=device))

    return model

In [18]:
# efficientnet
efficientnet = load_model('./ensemble/efficientnetb4.pth', 'EfficientnetB4', 18, device).to(device)
# efficientnet 2
efficientnet2 = load_model('./ensemble/efficientnetb4_2.pth', 'EfficientnetB4', 18, device).to(device)
# efficientnet 3
efficientnet3 = load_model('./ensemble/efficientnetb4_3.pth', 'EfficientnetB4', 18, device).to(device)
# efficientnet 4
efficientnet4 = load_model('./ensemble/efficientnetb4_4.pth', 'EfficientnetB4', 18, device).to(device)

Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4
Loaded pretrained weights for efficientnet-b4


In [19]:
efficientnet.eval()
efficientnet2.eval()
efficientnet3.eval()
efficientnet4.eval()

EfficientnetB4(
  (efficientnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )


In [20]:
data_dir = '/opt/ml/input/data/eval'

In [21]:
img_root = os.path.join(data_dir, 'images')
info_path = os.path.join(data_dir, 'info.csv')
info = pd.read_csv(info_path)

In [22]:
img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]
dataset = TestDataset(img_paths, (96, 128))
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=30,
    num_workers=1,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False,
)

In [23]:
output_dir = './ensemble'

In [24]:
print("Calculating inference results..")
preds = []
with torch.no_grad():
    for idx, images in enumerate(loader):
        images = images.to(device)
        pred_eff = efficientnet(images)
        pred_eff_2 = efficientnet2(images)
        pred_eff_3 = efficientnet3(images)
        pred_eff_4 = efficientnet4(images)
        pred = (pred_eff+pred_eff_2+pred_eff_3+pred_eff_4) / 4
        pred = pred.argmax(dim=-1)
        preds.extend(pred.cpu().numpy())

info['ans'] = preds
info.to_csv(os.path.join(output_dir, f'output.csv'), index=False)
print(f'Inference Done!')

Calculating inference results..
Inference Done!


In [25]:
pred_eff.argmax(dim=-1)

tensor([ 0,  3,  0,  2,  0,  3,  6,  6, 16,  2, 13, 15,  1, 10,  3,  0,  2,  6,
         2,  0,  6,  2,  2,  0,  0,  2,  5,  9,  1,  8], device='cuda:0')

In [26]:
pred_eff_2.argmax(dim=-1)

tensor([ 0,  3,  0,  2,  0,  3,  6,  6, 16,  2, 13, 15,  1, 10,  3,  0,  2,  6,
         2,  0,  6,  2,  2,  0,  0,  2,  5,  9,  1,  8], device='cuda:0')

In [27]:
pred_eff_3.argmax(dim=-1)

tensor([ 0,  3,  0,  2,  0,  3,  6,  6, 16,  2, 13, 15,  1, 10,  3,  0,  2,  6,
         2,  0,  6,  2,  2,  0,  0,  2,  5,  9,  1,  8], device='cuda:0')

In [28]:
pred_eff_4.argmax(dim=-1)

tensor([ 0,  3,  0,  2,  0,  3,  6,  6, 16,  2, 13, 15,  1, 10,  3,  0,  2,  6,
         2,  0,  6,  2,  2,  0,  0,  2,  5,  9,  1,  8], device='cuda:0')